# SF-133 Budget Execution

## Background

A handful of required DATA Act elements are found on the [Report on Budget Execution and Budgetary Resources (SF-133)](http://www.whitehouse.gov/sites/default/files/omb/assets/a11_current_year/s130.pdf).

This notebook examines the quarterly SF-133s published by OMB and matches them to the following DATA Act elements:
* Amount of budget authority appropriated
* Obligated amount
* Unobligated amount
* ~~Amount of other budgetary resources~~
* Outlay

This notebook **does not** attempt to recreate the SF-133 from individual agency account trial balance submissions (via the USSGL <--> SF-133 crosswalk). 

In [1]:
import pandas as pd
from pyquery import PyQuery as pq
from lxml import etree
from urllib.parse import urljoin
from slugify import slugify

## Get SF-133 Raw Data

OMB maintains copies of SF-133s on its public MAX portal:  
[Public MAX portal for Reports on Budget Execution and Budgetary Resources](https://max.omb.gov/maxportal/document/SF133/Budget/FY%202014%20-%20SF%20133%20Reports%20on%20Budget%20Execution%20and%20Budgetary%20Resources.html)

In [2]:
try:
    sf133 = pd.read_csv('data/sf133_all.csv')
except:
    pages = {
        '2013' : 'https://max.omb.gov/maxportal/document/SF133/Budget/FY%202013%20-%20SF%20133%20Reports%20on%20Budget%20Execution%20and%20Budgetary%20Resources.html',
        '2014' : 'https://max.omb.gov/maxportal/document/SF133/Budget/FY%202014%20-%20SF%20133%20Reports%20on%20Budget%20Execution%20and%20Budgetary%20Resources.html',
        '2015' : 'https://max.omb.gov/maxportal/document/SF133/Budget/FY%202013%20-%20SF%20133%20Reports%20on%20Budget%20Execution%20and%20Budgetary%20Resources.html'
    }
    #generate list of quarterly SF-133 URLs
    #(quarterly reports are .xls and monthly reports are .xlsx, 
    #which makes it easier to grab the right ones)
    urls = []
    for year in pages:
        html = pq(pages[year])
        links = html('a[href$="xls"]')
        for link in links:
            urls.append(urljoin(html.base_url, pq(link).attr['href']))

    #concatenate the quarterly SF-133s
    sf133 = pd.DataFrame()
    for url in urls:
        df = pd.read_excel(url, sheetname=0)
        print ('adding {} rows from {}'.format(len(df.index), url))
        sf133 = pd.concat([sf133, df])
    #cleanup column names
    sf133.columns = [slugify(
        x, to_lower = True, separator = '_') for x in sf133.columns.values]
    #save concatenated sf133
    sf133.to_csv('data/sf133_all.csv', index=False)
sf133.lno = sf133.lno.astype(str)

## SF133 <--> DATA Act Crosswalk

We've already done some work to match SF-133 lines to required
DATA Act elements. This information (along with some detailed mapping
info to the US General Ledger) is in a .csv file that ships with this notebook.

Most information in the file is related to the General Ledger mapping, so we'll
clean things up to get only what we want: the name of a DATA Act
element and it's corresponding line on the SF-133

In [3]:
crosswalk = pd.read_csv('data/data_act_gl_mapping.csv')
crosswalk.columns = [slugify(
        x, to_lower=True, separator='_') for x in crosswalk.columns.values]
crosswalk['da_element_number'], crosswalk['da_element_name'] = zip(
    *crosswalk['data_act_element'].apply(lambda x: x.split(' - ', 1)))
crosswalk = crosswalk[['da_element_number', 'da_element_name', 'sf_133_line_number']]
crosswalk.drop_duplicates(inplace=True)

Now we're left with a handful of DATA Act elements that can be
dervied from the SF-133

In [4]:
crosswalk


,da_element_number,da_element_name,sf_133_line_number
0,52,Amount of Budget Authority Appropriated,1900
420,53,Obligated Amount,2190
524,54,Unobligated Amount,2490
549,55,Amount of Other Budgetary Resources,need more info/definition
550,57,Outlay,4190


## Get DATA Act-Related Subset of SF-133

Create a subset of the SF-133 limited to DATA Act lines.

In [5]:
sf133_data_act = pd.merge(
    sf133, crosswalk,
    left_on = 'lno',
    right_on = 'sf_133_line_number'
)

## Explore the Data

(coming soon)